<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Colab" data-toc-modified-id="Colab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Colab</a></span></li><li><span><a href="#Load-the-libraries" data-toc-modified-id="Load-the-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the libraries</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Train-Test-Split" data-toc-modified-id="Train-Test-Split-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train Test Split</a></span></li></ul></div>

# Colab

In [1]:
%%capture
# capture will not print in notebook

import os
import sys
ENV_COLAB = 'google.colab' in sys.modules

if ENV_COLAB:
    ## install modules
    !pip install watermark
    !pip install scikit-plot
    !pip install -U scikit-learn # tweedie needs > 0.23

    ## print
    print('Environment: Google Colaboratory.')

# NOTE: If we update modules in gcolab, we need to restart runtime.

In [2]:
import scikitplot as skplot

In [3]:
TREE_METHOD = 'auto'
try:
    import tensorflow as tf
    has_gpu = tf.test.gpu_device_name()
    TREE_METHOD = 'gpu_hist' if has_gpu else 'auto'
except:
    TREE_METHOD = 'auto'

print(TREE_METHOD)

auto


In [4]:
if ENV_COLAB:
    !mkdir -p ../data/processed
    !mkdir -p ../data/raw
    !mkdir -p ../outputs

    # raw
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/raw/freMTPL2freq.csv?raw=true" -o ../data/raw/freMTPL2freq.csv
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/raw/freMTPL2sev.csv?raw=true" -o ../data/raw/freMTPL2sev.csv

    # processed
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/processed/clean_data.csv.zip?raw=true" -o ../data/processed/clean_data.csv.zip
    !wget -c "https://raw.githubusercontent.com/bhishanpdl/Datasets/master/Projects/French_Motor_Claims/processed/features.json" -o ../data/processed/features.json
    !wget -c "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/processed/X.npz?raw=true" -o ../data/processed/X.npz

In [5]:
if  ENV_COLAB:
    !ls ../data/raw
    !echo "processed"
    !ls ../data/processed

# Load the libraries

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import os,sys,time
import sklearn
import scipy
import matplotlib.pyplot as plt
sns.set()

import json
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_tweedie_deviance
from sklearn.metrics import auc

SEED = 100
pd.set_option('max_columns',100)
pd.set_option('plotting.backend','matplotlib') # matplotlib, bokeh, altair, plotly
%load_ext watermark
%watermark -iv

pandas     1.1.0
joblib     0.16.0
seaborn    0.11.0
numpy      1.18.4
sklearn    0.23.1
json       2.0.9
autopep8   1.5.2
scikitplot 0.3.7
scipy      1.4.1



# Load the data

In [8]:
ifile = '../data/processed/clean_data.csv.zip'
df = pd.read_csv(ifile, compression='zip')
print(df.shape)
df.head(2).append(df.tail(2))

(100000, 15)


,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,PurePremium,Frequency,AvgClaimAmount
0,0,0.10,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0
1,0,0.77,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0
99998,0,0.90,C,7,9,44,50,B1,Regular,191,R24,0.0,0.0,0.0,0.0
99999,0,0.90,E,4,12,53,50,B1,Regular,4116,R24,0.0,0.0,0.0,0.0


In [14]:
X = pd.read_csv("../data/processed/X.csv.zip",header=None,compression='zip')
X.shape

(100000, 1)

In [ ]:
df.head(2)

In [ ]:
np.array(X[0].todense())[0][-5:] # last elements of first row

In [ ]:
with open("../data/processed/features.json") as fi:
    json_features = json.load(fi)

In [ ]:
json_features.keys()

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test, X_train, X_test = train_test_split(df, X, random_state=SEED)

target = ['Frequency']

y_train = df_train[target].to_numpy().ravel()
y_test = df_test[target].to_numpy().ravel()

df_train.shape, df_test.shape, X_train.shape, X_test.shape